In [11]:
import pandas as pd
import altair as alt

dir = '../Data'

weather = pd.read_csv(f'{dir}/weather_clean.csv')
collisions = pd.read_csv(f'{dir}/collisions_clean.csv')

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [12]:
colores_hex = [
    '#a3ffd6',  # Verde agua intenso
    '#d69bf5',  # Púrpura intenso
    '#ff8080',  # Rojo intenso
    '#80ff80',  # Verde intenso
    '#80bfff',  # Azul intenso
    '#ffff66',  # Amarillo intenso
    '#ffcc66',  # Naranja intenso
    '#c9cba3',  # Púrpura intenso
    '#66cccc',  # Verde azulado intenso
    '#ff66b3',  # Rosa intenso
    '#ffb056',  # Verde claro intenso
    '#98c1d9',  # Rojo anaranjado intenso
    '#ffafcc'   # Verde intenso
]


In [13]:
# select the year from the CRASH DATE column
collisions['YEAR'] = collisions['CRASH DATE'].astype(str).str[:4]

## Is there any type of vehicle more prone to participate in accidents?

In [14]:
collisions["VEHICLE TYPE CODE 1"].unique()

array(['VAN', 'CAR', 'BUS', 'TRACTOR', 'TAXI', 'SCOOTER', 'TRUCK',
       'MOTORCYCLE', 'BICYCLE', 'OTHERS', 'AMBULANCE', 'FDNY', 'UNKNOWN'],
      dtype=object)

Given that we aggregated the type of vehicles to a total of just 13 classes, the bar chart is the first option that comes to our mind. 

In [15]:
df = collisions[['VEHICLE TYPE CODE 1']]

alt.Chart(df).mark_bar().encode(
    x=alt.X('VEHICLE TYPE CODE 1:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count():Q', title='Number of Collisions')
).properties(
    width=500,
    height=200,
    title='Number of Collisions by Vehicle Type'
).configure_mark(
    color='lightblue'
)

alt.Chart(...)

The result is decent but at the same time there are some classes barely noticeable. We want to clarify that the answer of this question is impossible with the available data, since we would need the traffic percenatge or proportion of every type of vehicle to determine if there is a vehicle more prone to have crashes than others. This happens because, for example, there are much more cars than ambulances, so the number of total car crashes is much bigger than the numbe of total ambulance crashes, and without the traffic proportions we can't really say if one of the vehicles is more prone to have an accident than the others. 

Having stated this, we thought that polar area charts could be a good option to make the classes with less crashes more noticeable. We are aware that the comparison between areas is more difficult, however we believe that the main (erroneous for the reasons previously explained) conclusions for the question are still clear enough and pretty understandable.

In [16]:
alt.Chart(collisions).encode(
    alt.Theta("VEHICLE TYPE CODE 1:N",
              stack = True,
              sort=alt.EncodingSortField(field="count", op="count", order='descending')),
    alt.Radius("count()",
               scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("VEHICLE TYPE CODE 1:N",
                    sort=alt.EncodingSortField(field="count", op="count", order='descending'),
                    scale=alt.Scale(range=colores_hex),
                    legend=alt.Legend(title="Vehicle Type")),
).mark_arc(
    innerRadius=5, stroke="#fff"
)

alt.Chart(...)

The choice of this palette with no colors being extremely similar or much more appealing than others allows for a good idenftification of the thirteen instances. The strength of the chart is taht allows to see the groups with less total crashes, so the user can compare the smaller classes aswell as the bigger ones. However, the problem is, as explained previously, that areas are much more difficult to compare than lenghts, so although the user can say which are bigger than others (since its ordered by size), he can't really quantify how much bigger are ones from others.

To solve this problem, we determined that the solution would be to aggregate the numbers with the actual counts of crashes to each of the instances. Which resulted in the following plot:

In [28]:
c1 = alt.Chart(collisions).encode(
    alt.Theta("VEHICLE TYPE CODE 1:N",
              stack = True,
              sort=alt.EncodingSortField(field="count", op="count", order='descending')),
    alt.Radius("count()",
               scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("VEHICLE TYPE CODE 1:N",
                    sort=alt.EncodingSortField(field="count", op="count", order='descending'),
                    scale=alt.Scale(range=colores_hex),
                    legend=alt.Legend(title="Vehicle Type")),
).mark_arc(
    innerRadius=5, stroke="#fff"
)

text = c1.mark_text(
    align='center',
    baseline='middle',
    radiusOffset=20,
    fontSize=12.5,
).encode(
    text='count()'
)

alt.layer(c1 + text).properties(
    title='Number of Collisions by Vehicle Type'
)

alt.LayerChart(...)

In [18]:
# create a new column that sums NUMBER OF PERSONS INJURED and NUMBER OF PERSONS KILLED
collisions['TOTAL INJURED/KILLED'] = collisions['NUMBER OF INJURED'] + collisions['NUMBER OF KILLED']

The last option we considered is a lollipop chart since it allows an easy comparison of several instances. The result is the following and it is similar to the one of a typical barchart. Simple and effective, but with the same problem of some types of vehicle being unnoticeable and impossible to differentiate (the ones with the lowest counts of crashes).

In [19]:
df = collisions[['VEHICLE TYPE CODE 1', 'TOTAL INJURED/KILLED']]

df1 = df.groupby('VEHICLE TYPE CODE 1').sum("TOTAL INJURED/KILLED").reset_index()
df2 = df.groupby('VEHICLE TYPE CODE 1').count().reset_index()

df2.columns = ['VEHICLE TYPE CODE 1', 'TOTAL COLLISIONS']

df = pd.merge(df1, df2, on='VEHICLE TYPE CODE 1')


lolli = alt.Chart(df).mark_bar(
    size=3
).encode(
    x=alt.X('TOTAL COLLISIONS:Q',
            title='Total Collisions'),
    y=alt.Y('VEHICLE TYPE CODE 1:N',
            title='Vehicle Type',
            sort=alt.EncodingSortField(field="TOTAL COLLISIONS", order="descending"),
            axis=alt.Axis(labelAngle=0, grid=True)),
    color=alt.Color('VEHICLE TYPE CODE 1:N',
                    title='Vehicle Type',
                    legend=None),
)

pop = alt.Chart(df).mark_circle(
    tooltip=True,
    size=80,
    opacity=1
).encode(
    x=alt.X('TOTAL COLLISIONS:Q',
            title='Total Collisions'),
    y=alt.Y('VEHICLE TYPE CODE 1:N',
            title='Vehicle Type',
            sort=alt.EncodingSortField(
                field="TOTAL COLLISIONS",
                order="descending"),
            axis=alt.Axis(labelAngle=0, grid=True)),
    color=alt.Color('VEHICLE TYPE CODE 1:N',
                    title='Vehicle Type',
                    legend=None),
    tooltip=['VEHICLE TYPE CODE 1:N', 'TOTAL COLLISIONS:Q', 'TOTAL INJURED/KILLED:Q']
).properties(
    title='Lollipop Plot of Collisions by Vehicle Type and Contributing Factor'
)

lolli + pop

alt.LayerChart(...)

We decided that the best option in this case is the last iteration of the polar area chart. It solves the problem of less common classes being really difficult to differentiate and offers the actual count values so that the user can get an idea of how much difference is between the number of accidents of the different vehicle types. A possible problem is that for this type of chart so many instances could be difficult to interpret at first glance, but since we have chosen a proper palette, ordered the instances by value and written their values, we consider this is just a minor problem that should not affect much the aim of the visualization.